In [485]:
import numpy as np

In [486]:
### initial data generation
deg = 3
### Samples per mode
Nspm = 100
### Number of modes
Nm = 3
###
Ns = Nm*Nspm

Mean = []
Cov = []
data = []

for i in range(Nm):
    Mean.append(np.random.randint(1,10,deg))
    temp = np.random.randint(2,9,(deg,deg))
    Cov.append(np.matmul(temp,temp.T))
    for j in range(Nspm):
        data.append(np.random.multivariate_normal(Mean[i],Cov[i],1))   
data = np.array(data)
data = data.reshape(Ns,deg)

In [487]:
def gaus_mul_pdf(x,mean,cov,degree):
    x = x.reshape((degree,1))
    mean = mean.reshape((degree,1))
    ph = np.matmul(np.matmul(((x-mean).T),np.linalg.inv(cov)),(x-mean))
    temp = np.exp((-0.5)*ph)/np.sqrt((2*np.pi*np.linalg.det(cov)))
    temp = np.asscalar(temp)
#     print(temp.shape)
    return temp

In [488]:
def gam(mix,data_matrix,mean_matrix,cov_matrix,Nm,i,k,degree):
    temp = 0
    for w in range(Nm):
        temp = temp + mix[w]*gaus_mul_pdf(data_matrix[i],mean_matrix[w],cov_matrix[w],degree)
    temp1 = mix[k]*gaus_mul_pdf(data_matrix[i],mean_matrix[k],cov_matrix[k],degree)/temp
    return temp1

In [489]:
def Nk(mix,data_matrix,mean_matrix,cov_matrix,Nm,k,Ns,degree):
    temp = 0
    for w in range(Ns):
        temp = temp + gam(mix,data_matrix,mean_matrix,cov_matrix,Nm,w,k,degree)
    return temp

In [490]:
def log_likelihood(data,mean_matrix,cov_matrix,mix,deg,Ns):
    temp = 0
    temp1 = 0
    for i in range(Ns):
        for j in range(deg):
            temp += mix[j]*gaus_mul_pdf(data[i],mean_matrix[j],cov_matrix[j],deg)
        temp1 += np.log(np.array(temp))
    print(temp1)
    return temp1

In [491]:
### Initial Parameter setting
# mix = np.full((1,Nm),1/Nm)
mix = []
cov_matrix = []
mean_matrix = []
for i in range(Nm):
    mean_matrix.append(np.random.randint(1,10,deg))
    temp = (np.random.randint(1,10,(deg,deg)))
    cov_matrix.append(np.matmul(temp,temp.T))
    mix.append(float(1.0/Nm))

In [492]:
# cov_matrix[0]
log_likelihood(data,mean_matrix,cov_matrix,mix,deg,Ns)

-918.6423309393222


-918.6423309393222

In [493]:
itera = 0
err_delta = 100.0
while(itera<100 or err_delta > 1e-5):
    new_mean_matrix = []
    new_cov_matrix = []
    new_mix = []
    log_o = log_likelihood(data,mean_matrix,cov_matrix,mix,deg,Ns)
    print(mix)
    for k in range(Nm):
        meh = Nk(mix,data,mean_matrix,cov_matrix,Nm,k,Ns,deg)
        temp1 = 0
        temp2 = 0
        temp3 = 0
        for i in range(Ns):
            temp1 = np.outer((data[i]-mean_matrix[k]),(data[i]-mean_matrix[k]))
            temp2 += gam(mix,data,mean_matrix,cov_matrix,Nm,i,k,deg)*temp1
            temp3 += gam(mix,data,mean_matrix,cov_matrix,Nm,i,k,deg)*data[i]
        new_cov_matrix.append(temp2/meh)
        new_mean_matrix.append(temp3/meh)
        new_mix.append(meh/Ns)
    mean_matrix = new_mean_matrix
    cov_matrix = new_cov_matrix
    mix = new_mix
    log_n = log_likelihood(data,mean_matrix,cov_matrix,mix,deg,Ns)
    err_delta = np.abs(log_o - log_n)
    print("### ",err_delta)
    itera += 1

-918.6423309393222
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
-734.1191692593435
('### ', 184.52316167997867)
-734.1191692593435
[0.1706751739100264, 0.13184173212525183, 0.6974830939647219]
-631.1932210101584
('### ', 102.92594824918513)
-631.1932210101584
[0.1317084270331201, 0.10111825020156447, 0.7671733227653157]
-626.890859590599
('### ', 4.302361419559361)
-626.890859590599
[0.11003066714256107, 0.10131069096769832, 0.7886586418897406]
-625.0095994599284
('### ', 1.8812601306706256)
-625.0095994599284
[0.09750753019647959, 0.10608915448562067, 0.7964033153179002]
-623.6541657076751
('### ', 1.3554337522532478)
-623.6541657076751
[0.08999982428571005, 0.1125922671628994, 0.7974079085513902]
-623.2672904480103
('### ', 0.3868752596648619)
-623.2672904480103
[0.08571901168381377, 0.11972166036137158, 0.7945593279548148]


KeyboardInterrupt: 